#### Faiss
Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning.

In [4]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter


In [5]:
#initalize the loader
loader=TextLoader("speech.txt")
text=loader.load()
text

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness be

In [6]:
#initialize the charcter text splitter
text_splitter=CharacterTextSplitter(chunk_size=2000,chunk_overlap=30)
docs=text_splitter.split_documents(text)
docs

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness be

In [ ]:
#instantiate the embedding model
embeddings= (
    OllamaEmbeddings(model="gemma2:2b")  # by default it uses llama 2
)

#save all the vectors into facebookai_similaritysearch vector db
db=FAISS.from_documents(docs,embeddings)


In [9]:
db

## Lets as k question ollama model 

In [11]:
##lets query the data out of vector db
query="How does the speaker describe the desired outcome of the war?"
docs=db.similarity_search(query)
docs[0]


Document(id='e31a1dec-675e-4eae-aea3-55c646bfa845', metadata={'source': 'speech.txt'}, page_content='We have borne with their present government through all these bitter months because of that friendship—exercising a patience and forbearance which would otherwise have been impossible. We shall, happily, still have an opportunity to prove that friendship in our daily attitude and actions toward the millions of men and women of German birth and native sympathy who live among us and share our life, and we shall be proud to prove it toward all who are in fact loyal to their neighbors and to the government in the hour of test. They are, most of them, as true and loyal Americans as if they had never known any other fealty or allegiance. They will be prompt to stand with us in rebuking and restraining the few who may be of a different mind and purpose. If there should be disloyalty, it will be dealt with with a firm hand of stern repression; but, if it lifts its head at all, it will lift it o

In [12]:
docs[0].page_content

'We have borne with their present government through all these bitter months because of that friendship—exercising a patience and forbearance which would otherwise have been impossible. We shall, happily, still have an opportunity to prove that friendship in our daily attitude and actions toward the millions of men and women of German birth and native sympathy who live among us and share our life, and we shall be proud to prove it toward all who are in fact loyal to their neighbors and to the government in the hour of test. They are, most of them, as true and loyal Americans as if they had never known any other fealty or allegiance. They will be prompt to stand with us in rebuking and restraining the few who may be of a different mind and purpose. If there should be disloyalty, it will be dealt with with a firm hand of stern repression; but, if it lifts its head at all, it will lift it only here and there and without countenance except from a lawless and malignant few.\n\nIt is a distr

## lets retrieve some info from the ollama model 

#### As a Retriever
We can also convert the vectorstore into a Retriever class. This allows us to easily use it in other LangChain methods, which largely work with retrievers

In [15]:
#intialise the db retriever
retriver=db.as_retriever()
docs =retriver.invoke(query)
docs

[Document(id='e31a1dec-675e-4eae-aea3-55c646bfa845', metadata={'source': 'speech.txt'}, page_content='We have borne with their present government through all these bitter months because of that friendship—exercising a patience and forbearance which would otherwise have been impossible. We shall, happily, still have an opportunity to prove that friendship in our daily attitude and actions toward the millions of men and women of German birth and native sympathy who live among us and share our life, and we shall be proud to prove it toward all who are in fact loyal to their neighbors and to the government in the hour of test. They are, most of them, as true and loyal Americans as if they had never known any other fealty or allegiance. They will be prompt to stand with us in rebuking and restraining the few who may be of a different mind and purpose. If there should be disloyalty, it will be dealt with with a firm hand of stern repression; but, if it lifts its head at all, it will lift it 

#### Similarity Search with score
There are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 distance. Therefore, a lower score is better. They use the Mahanttan distance 

In [16]:
docs_and_score=db.similarity_search_with_score(query)
docs_and_score


[(Document(id='e31a1dec-675e-4eae-aea3-55c646bfa845', metadata={'source': 'speech.txt'}, page_content='We have borne with their present government through all these bitter months because of that friendship—exercising a patience and forbearance which would otherwise have been impossible. We shall, happily, still have an opportunity to prove that friendship in our daily attitude and actions toward the millions of men and women of German birth and native sympathy who live among us and share our life, and we shall be proud to prove it toward all who are in fact loyal to their neighbors and to the government in the hour of test. They are, most of them, as true and loyal Americans as if they had never known any other fealty or allegiance. They will be prompt to stand with us in rebuking and restraining the few who may be of a different mind and purpose. If there should be disloyalty, it will be dealt with with a firm hand of stern repression; but, if it lifts its head at all, it will lift it

In [17]:
#passing vectors to Ollama

embedding_vector=embeddings.embed_query(query)
embedding_vector

[1.1934486627578735,
 -0.1075410395860672,
 -1.484168291091919,
 -0.9849861860275269,
 -0.6686685681343079,
 -0.3247118890285492,
 0.5366525053977966,
 0.7748734951019287,
 0.03189627081155777,
 -0.1502513438463211,
 -1.1376219987869263,
 -0.9146572351455688,
 -0.5137201547622681,
 0.08461645990610123,
 -3.3621907234191895,
 -2.330098867416382,
 -0.2972007393836975,
 -0.3820969760417938,
 -2.5682311058044434,
 -0.6716466546058655,
 0.555904746055603,
 -1.8029781579971313,
 -1.8973374366760254,
 1.4011991024017334,
 -2.500321626663208,
 -2.4635205268859863,
 1.552549123764038,
 -0.5822492837905884,
 -2.087622880935669,
 2.515347480773926,
 -3.143024206161499,
 -1.508427381515503,
 1.186556100845337,
 1.0656338930130005,
 1.7343510389328003,
 1.3721117973327637,
 -0.3412744402885437,
 0.57595294713974,
 3.0000813007354736,
 -3.5103816986083984,
 1.5240172147750854,
 0.37255939841270447,
 -1.484560489654541,
 -2.977181911468506,
 -3.553356647491455,
 -2.4893784523010254,
 -0.2804211974143

In [18]:
docs_score=db.similarity_search_by_vector(embedding_vector)
docs_score

[Document(id='e31a1dec-675e-4eae-aea3-55c646bfa845', metadata={'source': 'speech.txt'}, page_content='We have borne with their present government through all these bitter months because of that friendship—exercising a patience and forbearance which would otherwise have been impossible. We shall, happily, still have an opportunity to prove that friendship in our daily attitude and actions toward the millions of men and women of German birth and native sympathy who live among us and share our life, and we shall be proud to prove it toward all who are in fact loyal to their neighbors and to the government in the hour of test. They are, most of them, as true and loyal Americans as if they had never known any other fealty or allegiance. They will be prompt to stand with us in rebuking and restraining the few who may be of a different mind and purpose. If there should be disloyalty, it will be dealt with with a firm hand of stern repression; but, if it lifts its head at all, it will lift it 

In [ ]:
#Save the model to local
db.save_local("faiss_index")

In [22]:
# pull in the saved model
new_df=FAISS.load_local("faiss_index",embeddings, allow_dangerous_deserialization=True)
docs=new_df.similarity_search(query)